In [1]:
import requests
from kafka import KafkaProducer
import json

def send_data_from_url_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        feeds = data.get("feeds", [])

        # Publish each entry to Kafka
        for feed in feeds:
            for field in fields:
                feed[fields[field]] = feed.pop(field)
            producer.send(topic, value=feed)
            #print(f"Sent: {feed}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(feeds)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [ ]:
# https://open.tan.fr/doc/openapi#tag/Arrets/operation/getArretsHorairesTheoriques : 
fields = {
    "field1": "codeArret",
    "field2": "numLigne",
    "field3": "sens",
}

api_url = "https://openv2-preprod.tan.fr/ewp/horairesarret.json/{codeArret}/{numLigne}/{sens}"

# Code Arret 
codeArret = "HBLI2"

# Num Ligne
numLigne = "C5"

# Sens
sens = 1

In [5]:
send_data_from_url_to_kafka("test_topic", api_url.format(codeArret=codeArret, numLigne=numLigne, sens=sens), fields)

Failed to fetch data: 500
